# 通过 RPC 查询数据

通过单击顶部栏上的`🚀` -&gt; `Binder`在线运行此示例！

## 准备

In [1]:
# preparation: import and init w3 instance
from conflux_web3 import Web3

w3 = Web3(Web3.HTTPProvider("https://test.confluxrpc.com"))

# prepare constants
transaction_hash = "0x10a721e2654523a7ff682c1c8d3f868a9fdf78a3be1858bbe1f06147128d6d94"
epoch = 98943500

## 通过 RPC 查询

我们可以使用 SDK 调用 RPC 调用来查询区块链状态交易数据、区块数据等。完整的 RPC 文档可以在[这里](https://developer.confluxnetwork.org/conflux-doc/docs/json_rpc/#json-rpc-methods)找到。

### 查询区块链状态

In [2]:
# get_status provides an overview of blockchain status
w3.cfx.get_status()

AttributeDict({'bestHash': HexBytes('0x7402986f3f8116515e4cfff0df2941d105272158825c55f48cf8c6b1bcf55d0b'),
 'chainId': 1,
 'ethereumSpaceChainId': 71,
 'networkId': 1,
 'epochNumber': 98947195,
 'blockNumber': 126750704,
 'pendingTxNumber': 0,
 'latestCheckpoint': 98860000,
 'latestConfirmed': 98947131,
 'latestState': 98947191,
 'latestFinalized': 98946660})

几个 RPC 方法可以接受一个**epoch number 参数**。纪元数参数可以是：

- 一个整数
- 或者一个时代标签`latest_mined` , `latest_state` , `latest_confirmed`或者`latest_finalized`

Conflux 中的`epoch number`的概念有点类似于其他区块链中的`block number`的概念，但一个 epoch 包含一个或多个区块。有关详细信息，请参阅 https://developer.confluxnetwork.org/conflux-doc/docs/json_rpc#the-default-epochnumber-parameter。

In [3]:
# latest_mined epoch number
print(f"latest_mined epoch number: {w3.cfx.epoch_number}")
# get epoch number by tag
print(f'latest_state epoch numebr: {w3.cfx.epoch_number_by_tag("latest_state")}')

latest_mined epoch number: 98947195
latest_state epoch numebr: 98947191


### 查询交易

In [4]:
# get transaction data by hash
# this RPC is usable after transaction is sent
w3.cfx.get_transaction_by_hash(transaction_hash)

AttributeDict({'hash': HexBytes('0x10a721e2654523a7ff682c1c8d3f868a9fdf78a3be1858bbe1f06147128d6d94'),
 'nonce': 2,
 'blockHash': HexBytes('0xf0ac5f8d757a7415876e322a245d5eff76349b656b3eadb10d5e09a4a3f7d59d'),
 'transactionIndex': 0,
 'from': 'cfxtest:aan2vyszrgz3sr3tug73ywb8k128y5wa7yhpjm1hn1',
 'to': 'cfxtest:acejjfa80vj06j2jgtz9pngkv423fhkuxj786kjr61',
 'value': 0 Drip,
 'gasPrice': 1000000000 Drip,
 'gas': 41633,
 'contractCreated': None,
 'data': HexBytes('0x5f3cc9f7'),
 'storageLimit': 0,
 'epochHeight': 87083312,
 'chainId': 1,
 'status': 0,
 'v': 1,
 'r': HexBytes('0x17b30481d63a49223092215a2aa503938519df5cac0d20cbff7ebf50569776cd'),
 's': HexBytes('0x5fc96a1e383879cf66c09d106b2df4907c674a263c9a7bba602078c90b0038b1')})

In [5]:
# get transaction receipt by hash
# the transaction receipt is available after it is executed
w3.cfx.get_transaction_receipt(transaction_hash)

AttributeDict({'transactionHash': HexBytes('0x10a721e2654523a7ff682c1c8d3f868a9fdf78a3be1858bbe1f06147128d6d94'),
 'index': 0,
 'blockHash': HexBytes('0xf0ac5f8d757a7415876e322a245d5eff76349b656b3eadb10d5e09a4a3f7d59d'),
 'epochNumber': 87083316,
 'from': 'cfxtest:aan2vyszrgz3sr3tug73ywb8k128y5wa7yhpjm1hn1',
 'to': 'cfxtest:acejjfa80vj06j2jgtz9pngkv423fhkuxj786kjr61',
 'gasUsed': 35480,
 'gasFee': 35480000000000 Drip,
 'contractCreated': None,
 'logs': [],
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'

### 查询块

在 Conflux 中，每个 epoch 包含 1 个或多个块，其中一个块称为`pivot`块。枢轴块确定哪些块处于特定时期。

In [6]:
# query blocks in specific epoch
block_hashes = w3.cfx.get_blocks_by_epoch(epoch)
print(f"blocks in epoch {epoch}:\n{block_hashes}")
# the last element of block_hashes is the pivot block
print(f"pivot block hash of epoch {epoch}: {block_hashes[-1].hex()}")

blocks in epoch 98943500:
[HexBytes('0xae622377e139608a9202504df4bf20570c4741370d36f668952ae91c57c5da69'), HexBytes('0x104a3549108c7c83a36c65d188e842f6deb9a770d3ca749a222fce628846c04b')]
pivot block hash of epoch 98943500: 0x104a3549108c7c83a36c65d188e842f6deb9a770d3ca749a222fce628846c04b


In [7]:
# if the block is a pivot block, we can get it by using epoch number
assert w3.cfx.get_block_by_epoch_number(epoch) == w3.cfx.get_block_by_hash(block_hashes[-1])

In [8]:
# get_block_by_epoch_number also accepts an epoch number tag
w3.cfx.get_block_by_epoch_number("latest_state")

AttributeDict({'hash': HexBytes('0x23e4589f00f86dc0eef1f268b8db567ad5d19a164597648c828fe266a4844081'),
 'parentHash': HexBytes('0xd3ebea17481278ba7f705f1a53d7a09ad45d907f34a23fad9a313c02213d515d'),
 'height': 98947193,
 'miner': 'cfxtest:aaskvgxcfej371g4ecepx9an78ngrke5ay9f8jtbgg',
 'deferredStateRoot': HexBytes('0xd97426e7ab18195c40fec710cdd649bd262de6950a081d3e6ad7d8096bd63bc7'),
 'deferredReceiptsRoot': HexBytes('0x12af19d53c378426ebe08ad33e48caf3efdaaade0994770c161c0637e65a6566'),
 'deferredLogsBloomHash': HexBytes('0xd397b3b043d87fcd6fad1291ff0bfd16401c274896d8c63a923727f077b8e0b5'),
 'blame': 0,
 'transactionsRoot': HexBytes('0xc5d2460186f7233c927e7db2dcc703c0e500b653ca82273b7bfad8045d85a470'),
 'epochNumber': 98947193,
 'blockNumber': 126750701,
 'gasLimit': 30000000,
 'gasUsed': 0,
 'timestamp': 1667980575,
 'difficulty': 56382505,
 'powQuality': HexBytes('0x084c7fe8'),
 'refereeHashes': [],
 'adaptive': False,
 'nonce': HexBytes('0x2f63f70ee09c76d7'),
 'size': 0,
 'custom': [H

In [9]:
# get a block by block number is also viable 
w3.cfx.get_block_by_block_number(110979562)

AttributeDict({'hash': HexBytes('0xf0ac5f8d757a7415876e322a245d5eff76349b656b3eadb10d5e09a4a3f7d59d'),
 'parentHash': HexBytes('0xeaf2f6227c068ae27cb847657af7eb7e3cfd6696f5a795d2f38159074d20a8c9'),
 'height': 87083316,
 'miner': 'cfxtest:aaskvgxcfej371g4ecepx9an78ngrke5ay9f8jtbgg',
 'deferredStateRoot': HexBytes('0xa26347f02f312db0fba655c29b9fa853e8e6242449a67added74c62679122f62'),
 'deferredReceiptsRoot': HexBytes('0x09f8709ea9f344a810811a373b30861568f5686e649d6177fd92ea2db7477508'),
 'deferredLogsBloomHash': HexBytes('0xd397b3b043d87fcd6fad1291ff0bfd16401c274896d8c63a923727f077b8e0b5'),
 'blame': 0,
 'transactionsRoot': HexBytes('0xdc62bb0e7115a9727688811f4e090efdfd203a516953ff7829e21dea29e1f166'),
 'epochNumber': 87083316,
 'blockNumber': 110979562,
 'gasLimit': 30000000,
 'gasUsed': 35480,
 'timestamp': 1660093852,
 'difficulty': 54757256,
 'powQuality': HexBytes('0x0365f39d'),
 'refereeHashes': [],
 'adaptive': False,
 'nonce': HexBytes('0xe72421314ac723e6'),
 'size': 113,
 'custo

### 其他 RPC

除此之外， `conflux-web3`支持 cfx 命名空间下的所有 rpc 方法。您可以访问 https://developer.confluxnetwork.org/conflux-doc/docs/json_rpc#json-rpc-methods 了解更多信息。

这里有些例子：

In [10]:
w3.cfx.gas_price

1000000000 Drip

In [ ]:
w3.cfx.get_code("cfxtest:acejjfa80vj06j2jgtz9pngkv423fhkuxj786kjr61")